# **Soni - do**
## **Generating Music with Machine Learning**


#### Author: Sonia Cobo
#### Date: July 2021

### Though this project doesn't have a hypothesis per se, it was done to kind off prove how AI has advanced and it is now able to generate music which has been associated with emotions and human capabilities for a long period of time.

In [12]:
# data augmentation - dividir canciones, modificarlas para tener mas datos

# Data

### The input to the model will be a series of notes from a MIDI file. MIDI (Musical Instrument Digital Interface) is a technical standard that describes a communications protocol, digital interface, and electrical connectors that connect a wide variety of electronic musical instruments and computers. They don't contain actual audio data and are small in size. They explain what notes are played, when they're played, and how long or loud each note should be.

### To keep the project simple only files with one instrument were chosen, in this case the instrument is piano and the type of songs is classical. 
### These songs have been obtained from the following datasets: http://www.piano-midi.de/ and https://www.mfiles.co.uk/classical-midi.htm


In [13]:
# no descargardas aun: https://github.com/Skuldur/Classical-Piano-Composer/tree/master/midi_songs
# https://drive.google.com/file/d/1qnQVK17DNVkU19MgVA4Vg88zRDvwCRXw/view

### Import all libraries

In [705]:
# data manipulation
import numpy as np
import pandas as pd 
from random import randint

# manipulate midi files
import glob
from music21 import *
#from music21 import converter, instrument, note, chord, meter, stream, duration, corpus
import pygame

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# route files
import os
import sys

# ml model
import pickle

import tensorflow as tf
from tensorflow import keras

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization 
from keras.callbacks import ModelCheckpoint
from keras.layers import Reshape


In [15]:
len(tf.config.experimental.list_physical_devices('GPU'))

0

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7525799484492965909
]


### Paths

In [483]:
# The route of this file is added to the sys path to be able to import/export functions
sep = os.sep
def route (steps):
    """
    This function appends the route of the file to the sys path
    to be able to import files from/to other foders within the EDA project folder.
    """
    route = os.getcwd()
    for i in range(steps):
        route = os.path.dirname(route)
    sys.path.append(route)
    return route

In [484]:
# paths

# path to raw data
path = route(1) + sep + "data" + sep + "raw_data" + sep
# path to data in the right key
path_1 = route(1) + sep + "data" + sep + "converted_data" + sep
# path to compiled notes list
path_2 = route(1) + sep + "data" + sep + "notes" + sep
# path to generated models
path_3 = route(1) + sep + "models" + sep
# path to generated midi files
path_4 = route(1) + sep + "reports" + sep

## Midi file exploration

Hablar de frecuencia y la transpuesta de fourier

In [485]:
# All information from the midi file (i.e. notes, pitch, chord, time signature, etc) is contained within the component list
def info_midi (path):
    """
    It returns all midi file information given its path

    """
    file = converter.parse(path)
    components = []
    for element in file.recurse():  
        components.append(element)
    return components

components = info_midi(path + "alb_esp1.mid")
#components

### Now that the midi file has been studied and its structure is known, data will be split into two object types: notes, rests and chords. 

### Note objects contain information about the pitch, octave, and offset of the note.
### Pitch refers to the frequency of the sound, or how high or low it is and is represented with the letters [A, B, C, D, E, F, G].
### Octave refers to which set of pitches you use on a piano.
### Offset refers to where the note is located in the piece.
### Rests are the silences in the piece.
### Chord objects are a set of notes that are played at the same time.


### Songs are transposed into C major and A minor key to ease predictions

## Data preparation

### Relevant information from midi file is encoded and saved into an array.

### We append the pitch of every note object using its string notation since the most significant parts of the note can be recreated using the string notation of the pitch. And we append every chord by encoding the id of every note in the chord together into a single string, with each note being separated by a dot. 

In [486]:
# Each midi file contains notes and chords. These two properties will be the input and output of the LSTM network so 
# they need to be taken out from all midi files. 

def get_notes_per_song(path, filename):
    """
    This function extracts all the notes, rests and chords from one midi file
    and saves it in a list in the converted_data folder.

    Param: Path of the midi file, filename (str)
    """
    components = info_midi(path + filename)
    note_list = []
    
    for element in components:
        # note pitches are extracted
        if isinstance(element, note.Note):
            note_list.append(str(element.pitch))
        # chords are extracted
        elif isinstance(element, chord.Chord):
            note_list.append(".".join(str(n) for n in element.normalOrder))    
        # rests are extracted
        elif isinstance(element, note.Rest):
            note_list.append("NULL")    #further transformation needs this value as str rather than np.nan

    with open(path_2 + "notes", "wb") as filepath:
        pickle.dump(note_list, filepath)
    
    return note_list

In [487]:
note_list = get_notes_per_song(path_1, "C_alb_esp1.mid")

In [578]:
# Load notes and chords previously separated
def load_notes (path, filename):
    """
    Load the note list containing pitches, rests and chords.
    
    Param: Path of the saved note list, and its name as string
    """
    with open(path + filename, "rb") as f:
        loaded_notes = pickle.load(f)
        return loaded_notes

note_list = load_notes(path_2, "notes_chopin")
#note_list

### The model will be first trained with a small proportioned of the songs to expedite time. Once the model is tunned properly all songs will be passed to improve its training.

### Now that all notes, rests and chords are in a list, these will be transformed from categorical data to integer-based numerical data. It is necessary to create input sequences for the network and their respective outputs. The output for each input sequence will be the first note or chord that comes after the sequence of notes in the input sequence in our list of notes.

In [579]:
len(note_list)

5354

In [580]:
def prepare_sequences(notes, sequence_length, step):
    """ 
    Prepare the sequences used by the neural network 

    """
    
    # get all pitchnames
    pitchnames = sorted(set(notes))
    print('Total unique notes:', len(pitchnames))

    # create a dictionary to convert pitches (strings) to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))  # rests are included  

    network_input = []
    network_output = []

    #sequence_in = []
    #sequence_out = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, step):    
        network_input.append(notes[i:i + sequence_length])
        network_output.append(notes[i + sequence_length])
        # exchange their values for their integer-code

        # network_input.append([note_to_int[elem] for elem in sequence_in])
        # network_output.append(note_to_int[sequence_out])

    x = np.zeros((len(network_input), sequence_length, len(pitchnames)))
    y = np.zeros((len(network_input), len(pitchnames)))
    for i, sequence in enumerate(network_input):
        for j, note in enumerate(sequence):
            x[i, j, note_to_int[note]] = 1
        y[i, note_to_int[network_output[i]]] = 1

    # n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    # network_input = np.reshape(network_input, (n_patterns, sequence_length, 1)) 
    # normalize input
    # network_input = network_input / float(len(set(notes)))  

    # network_output = np_utils.to_categorical(network_output) # used to convert array of labeled data to one-hot vector

    return x, y

### The length of each sequence will be 100 notes/chords for now. This means that to predict the next note in the sequence the network has the previous 100 notes to help make the prediction

In [581]:
x, y = prepare_sequences(notes=note_list, sequence_length=100, step=3)  # length y step pueden variar  


Total unique notes: 169


In [582]:
print(x.shape)
print(y.shape)

(1752, 100, 169)
(1752, 169)


In [561]:
def generate_notes(model, temperature=1.0):
    """ 
    Generate notes from the neural network based on a sequence of notes 
    """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(note_list)-100-1)

    pitchnames = sorted(set(note_list))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames)) 
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = note_list[start: (start+100)] 
    prediction_output = []
    patterns = []

    # generate 500 notes, roughly two minutes of music
    for note_index in range(5):
        prediction_input = np.zeros((1, 100, len(pitchnames)))
        for j, note in enumerate(pattern):
            prediction_input[0, j, note_to_int[note]] = 1.0
        preds = model.predict(prediction_input, verbose=0)[0]   #[0]?
        next_index = sample(preds, temperature=temperature)
        next_note = int_to_note[next_index]

        pattern = pattern[1:]
        pattern.append(next_note)

        prediction_output.append(next_note)

        patterns.append(next_index)
        #patterns = patterns[1:len(patterns)]

    return prediction_output, patterns, next_index, pattern

In [562]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Creation of the model

There are four different types of layers:

LSTM layers is a Recurrent Neural Net layer that takes a sequence as an input and can return either sequences (return_sequences=True) or a matrix.

Dropout layers are a regularisation technique that consists of setting a fraction of input units to 0 at each update during the training to prevent overfitting. The fraction is determined by the parameter used with the layer.

Dense layers or fully connected layers is a fully connected neural network layer where each input node is connected to each output node.

The Activation layer determines what activation function our neural network will use to calculate the output of a node.

In [710]:
def generator_model(latent_dim=(x.shape[1], x.shape[2])):

    model = Sequential()
    model.add(LSTM(512, input_shape=latent_dim, return_sequences=True))
    model.add(Dense(latent_dim[1]))
    model.add(Activation("softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

    return model

In [711]:
g = generator_model()
g.summary()

Model: "sequential_78"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_68 (LSTM)               (None, 100, 512)          1396736   
_________________________________________________________________
dense_57 (Dense)             (None, 100, 169)          86697     
_________________________________________________________________
activation_22 (Activation)   (None, 100, 169)          0         
Total params: 1,483,433
Trainable params: 1,483,433
Non-trainable params: 0
_________________________________________________________________


In [565]:
def prepare_sequences_gan(notes, sequence_length, step):
    """ 
    Prepare the sequences used by the neural network 

    """
    
    # get all pitchnames
    pitchnames = sorted(set(notes))
    print('Total unique notes:', len(pitchnames))

    # create a dictionary to convert pitches (strings) to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))  # rests are included  

    network_input = []
    network_output = []

    #sequence_in = []
    #sequence_out = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - 2*sequence_length, step):    
        network_input.append(notes[i:i + sequence_length])
        network_output.append(notes[i + sequence_length : i + 2*sequence_length])
        # exchange their values for their integer-code

        # network_input.append([note_to_int[elem] for elem in sequence_in])
        # network_output.append(note_to_int[sequence_out])

    x = np.zeros((len(network_input), sequence_length, len(pitchnames)))
    y = np.zeros((len(network_input), sequence_length, len(pitchnames)))
    for i, sequence in enumerate(network_input):
        for j, note in enumerate(sequence):
            x[i, j, note_to_int[note]] = 1
            y[i, j, note_to_int[network_output[i][j]]] = 1

    return x, y

In [553]:
import gc
gc.collect()

3498

In [554]:
x_gan, y_gan = prepare_sequences_gan(notes=note_list, sequence_length=100, step=3)  # length y step pueden variar  


Total unique notes: 169


In [566]:
g.fit(x_gan, y_gan, epochs=10)  


Epoch 1/10
54/54 [==============================] - 27s 466ms/step - loss: 4.4912
Epoch 2/10
54/54 [==============================] - 25s 457ms/step - loss: 4.1768
Epoch 3/10
54/54 [==============================] - 25s 466ms/step - loss: 3.9955
Epoch 4/10
54/54 [==============================] - 25s 464ms/step - loss: 3.7924
Epoch 5/10
54/54 [==============================] - 25s 462ms/step - loss: 3.4880
Epoch 6/10
54/54 [==============================] - 25s 464ms/step - loss: 3.2052
Epoch 7/10
54/54 [==============================] - 25s 466ms/step - loss: 2.8429
Epoch 8/10
54/54 [==============================] - 25s 469ms/step - loss: 2.5782
Epoch 9/10
54/54 [==============================] - 25s 470ms/step - loss: 2.3153
Epoch 10/10
54/54 [==============================] - 25s 470ms/step - loss: 2.0487


In [567]:
def generate_notes_gan(model, temperature=1.0):
    """ 
    Generate notes from the neural network based on a sequence of notes 
    """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(note_list)-100-1)

    pitchnames = sorted(set(note_list))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames)) 
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = note_list[start: (start+100)] 
    prediction_output = []
    patterns = []

    # generate 500 notes, roughly two minutes of music

    prediction_input = np.zeros((1, 100, len(pitchnames)))
    for j, note in enumerate(pattern):
        prediction_input[0, j, note_to_int[note]] = 1.0
    preds = model.predict(prediction_input, verbose=0)[0]   #[0]?
    
    for elem in list(preds):
        next_index = sample(elem, temperature=temperature)
        next_note = int_to_note[next_index]

        #pattern = pattern[1:]
        #pattern.append(next_note)

        prediction_output.append(next_note)

        patterns.append(next_index)
        #patterns = patterns[1:len(patterns)]

    return prediction_output, patterns, next_index, pattern

In [568]:
prediction_output, patterns, next_index, pattern = generate_notes_gan(g, temperature=1)
print(prediction_output)

['NULL', '3', '2.7', 'B-2', 'G3', 'D3', 'E-3', 'B-2', 'E-2', 'F3', '0.3', 'F3', '0.3', 'F3', '0.3', 'NULL', 'F2', 'F3', 'G3', 'NULL', 'B-2', 'B-2', 'NULL', 'NULL', '10.3', '10.3', 'E-2', 'E-2', 'B-1', 'NULL', 'E-3', 'NULL', '8', '11', 'NULL', '3.7.10', 'B-2', 'E-1', 'NULL', 'E-1', 'NULL', 'G#1', 'NULL', 'C#2', '9.11.2.5', 'G#1', 'NULL', 'G#2', 'NULL', '3.6.9', 'NULL', '10.1.5', 'NULL', '3.6.9', '1.5.8', '1.5.8', 'NULL', '7.0', 'G#2', '10.1.5', '1.5.8', 'NULL', 'C#2', '7', 'NULL', 'E-2', '7', 'NULL', 'NULL', 'NULL', '1', 'NULL', 'NULL', 'NULL', '5.8.0', '5.8.0', '1', '3', '8.0.3', 'C3', '2.5', '6.9.0', 'NULL', 'E-3', '2.5', 'C3', 'E-6', 'B2', 'C#4', '1', 'C#4', 'C4', 'C4', 'C4', 'B-3', 'C4', 'C4', 'E4', 'G#4', 'C4']


In [517]:
len(prediction_output)

100

In [569]:
def create_midi(prediction_output, patterns, path):
    """ convert the output from the prediction to notes and create a midi file from the notes"""
    
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif ("NULL" in pattern):
            new_rest = note.Rest(pattern)
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)   
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write("midi", fp= path + "test_output_11_gmodel_chopin.mid")   # first output 01/07/2021

    return midi_stream

In [570]:
create_midi = create_midi(prediction_output, patterns, path_4)


In [571]:
def play_music(music_file):
    """
    Play music given a midi file path
    """
    import music21
    try:
        # allow to stop the piece 
        pygame.mixer.init()
        clock = pygame.time.Clock() 
        pygame.mixer.music.load(music_file)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            # check if playback has finished
            clock.tick(10)

        freq = 44100    # audio CD quality
        bitsize = -16   # unsigned 16 bit
        channels = 2    # 1 is mono, 2 is stereo
        buffer = 1024    # number of samples
        pygame.mixer.init(freq, bitsize, channels, buffer)

    except KeyboardInterrupt:
        while True:
            action = input('Enter Q to Quit, Enter to Skip.').lower()
            if action == 'q':
                pygame.mixer.music.fadeout(1000)
                pygame.mixer.music.stop()
            else:
                break

In [573]:
# Plays music when the cell is executed 

play_music(path_4 + "test_output_11_gmodel_chopin.mid")

In [583]:
print(x.shape)
print(y.shape)

(1752, 100, 169)
(1752, 169)


In [ ]:
# hasta aquí va!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [648]:
def generate_real_samples(x):
    """
    Load and prepare training notes
    """
    # choose random instances
    start = np.random.randint(0, len(x)-100-1)
    # retrieve selected images
    x_real = x[start: (start+100)] 
    # generate 'real' class labels (1)
    y_real = np.ones((x.shape[2], 1))

    return x_real, y_real

In [649]:
x_real, y_real = generate_real_samples(x)

In [748]:
def generate_latent_points(note_list, x_real, x):

    pitchnames = sorted(set(note_list))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # create random matrix of numbers 
    noise = np.random.choice(len(pitchnames)-1, (x_real.shape[0], x.shape[1], x.shape[2]))  #-------

    x_latent = []
    # transform random int to note
    for elem in noise[0][0]:
        fake = int_to_note[abs(int(elem))]
        x_latent.append(fake)  

    #x_latent = np.array(np.random.shuffle(x_latent,(x_real.shape[0], x.shape[1], x.shape[2])))

    x_latent = np.zeros((x.shape[0], x.shape[1], x.shape[2])))
    for i, sequence in enumerate(network_input):
        for j, note in enumerate(sequence):
            x[i, j, int_to_note[note]] = 1

    return x_latent


In [749]:
x_latent = generate_latent_points(note_list, x_real, x)
x_latent.shape

TypeError: shuffle() takes exactly one argument (2 given)

In [730]:
def generate_fake_data(note_list, x_real, x, g_model):
	# generate points in latent space
	x_fake = generate_latent_points(note_list, x_real, x)
	# predict outputs
	x_fake = g_model.predict(x_fake)
	# create 'fake' class labels (0)
	y_fake = np.zeros((x.shape[2], 1))
	return x_fake, y_fake

In [724]:
x_fake, y_fake = generate_fake_data(note_list, x_real, x, g)
x_fake.shape

ValueError: in user code:

    C:\Users\Usuario\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training.py:1544 predict_function  *
        return step_function(self, iterator)
    C:\Users\Usuario\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training.py:1527 run_step  *
        outputs = model.predict_step(data)
    C:\Users\Usuario\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training.py:1500 predict_step  *
        return self(x, training=False)
    C:\Users\Usuario\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\base_layer.py:989 __call__  *
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    C:\Users\Usuario\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\input_spec.py:212 assert_input_compatibility  *
        raise ValueError('Input ' + str(input_index) + ' of layer ' +

    ValueError: Input 0 of layer sequential_78 is incompatibl

In [695]:
# define the standalone discriminator model
def discriminator_model(n_inputs=(x.shape[1], x.shape[2])):
	model = Sequential()
	model.add(LSTM(512, input_shape=n_inputs))
	model.add(Dense(x.shape[2]))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [696]:
d = discriminator_model()
d.summary()

Model: "sequential_72"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_66 (LSTM)               (None, 512)               1396736   
_________________________________________________________________
dense_54 (Dense)             (None, 169)               86697     
_________________________________________________________________
dense_55 (Dense)             (None, 1)                 170       
Total params: 1,483,603
Trainable params: 1,483,603
Non-trainable params: 0
_________________________________________________________________


In [712]:
def gan_model(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	model.add(BatchNormalization())
	# add the discriminator
	model.add(d_model)
	# compile model
	model.compile(loss='binary_crossentropy', optimizer="adam")
	return model

In [713]:
gan_model = gan_model(g, d)
gan_model.summary()

Model: "sequential_79"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_78 (Sequential)   (None, 100, 169)          1483433   
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 169)          676       
_________________________________________________________________
sequential_72 (Sequential)   (None, 1)                 1483603   
Total params: 2,967,712
Trainable params: 1,483,771
Non-trainable params: 1,483,941
_________________________________________________________________


In [ ]:
# First, the discriminator model is updated for a half batch of real samples, then a half batch of fake samples, 
# together forming one batch of weight updates. The generator is then updated via the combined GAN model. 
# Importantly, the class label is set to 1 or real for the fake samples. This has the effect of updating the generator toward 
# getting better at generating real samples on the next batch.

In [ ]:
# train the generator and discriminator
dataset = generate_real_samples(x)

def train(g_model, d_model, gan_model, x_real, dataset, latent_dim, n_epochs=10, n_batch=128):
	bat_per_epo = int(x_real / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			x_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(x_real, y_real)
			# generate 'fake' examples
			x_fake, y_fake = generate_fake_data(note_list, x_real, x)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(x_fake, y_fake)

			# prepare points in latent space as input for the generator
			x_gan = latent(dataset, half_batch)
			# create inverted labels for the fake samples
			y_gan = np.ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(x_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		#if (i+1) % 10 == 0:
		#	summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:
train(g, d, gan_model, x_real, dataset, latent_dim, n_epochs=100, n_batch=128)

In [14]:
model.fit(x, y, epochs=10)#, batch_size=128)

Epoch 1/10
7/7 [==============================] - 14s 375ms/step - loss: 4.0040
Epoch 2/10
7/7 [==============================] - 3s 374ms/step - loss: 3.3438
Epoch 3/10
7/7 [==============================] - 3s 372ms/step - loss: 3.2140
Epoch 4/10
7/7 [==============================] - 3s 369ms/step - loss: 3.0658
Epoch 5/10
7/7 [==============================] - 3s 375ms/step - loss: 3.0402
Epoch 6/10
7/7 [==============================] - 3s 372ms/step - loss: 2.9484
Epoch 7/10
7/7 [==============================] - 3s 370ms/step - loss: 3.0115
Epoch 8/10
7/7 [==============================] - 3s 376ms/step - loss: 2.9082
Epoch 9/10
7/7 [==============================] - 3s 369ms/step - loss: 3.1540
Epoch 10/10
7/7 [==============================] - 3s 368ms/step - loss: 2.8951


In [15]:
# save the model
model.save(path_3 + "model_5.h5")

In [16]:
# load the model 
model_4 = tf.keras.models.load_model(path_3 + "model_5.h5")


In [17]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [236]:
def generate_notes(model, x, temperature=1.0):
    """ 
    Generate notes from the neural network based on a sequence of notes 
    """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(note_list)-100-1)

    pitchnames = sorted(set(note_list))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames)) 
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = note_list[start: (start+100)] # antes tenia x que es menor que note_list por lo que algunos valores start se salian de la lista
    prediction_output = []
    patterns = []

    # generate 500 notes, roughly two minutes of music
    for note_index in range(5):
        prediction_input = np.zeros((1, 100, len(pitchnames)))
        for j, note in enumerate(pattern):
            prediction_input[0, j, note_to_int[note]] = 1.0
        preds = model.predict(prediction_input, verbose=0)[0]   #[0]?
        next_index = sample(preds, temperature=temperature)
        next_note = int_to_note[next_index]

        pattern = pattern[1:]
        pattern.append(next_note)

        prediction_output.append(next_note)

        patterns.append(next_index)
        #patterns = patterns[1:len(patterns)]

    return prediction_output, patterns, next_index, pattern

In [237]:
prediction_output, patterns, next_index, pattern = generate_notes(model, x, temperature=1)
print(prediction_output)

['G#3', 'A3', 'F3', 'G3', 'D4']


# Output

In [56]:
def create_midi(prediction_output, patterns, path):
    """ convert the output from the prediction to notes and create a midi file from the notes"""
    
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif ("NULL" in pattern):
            new_rest = note.Rest(pattern)
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)   
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write("midi", fp= path + "test_output_9.mid")   # first output 01/07/2021

    return midi_stream

In [57]:
create_midi = create_midi(prediction_output, patterns, path_4)


In [250]:
def play_music(music_file):
    """
    Play music given a midi file path
    """
    import music21
    try:
        # allow to stop the piece 
        pygame.mixer.init()
        clock = pygame.time.Clock() 
        pygame.mixer.music.load(music_file)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            # check if playback has finished
            clock.tick(10)

        freq = 44100    # audio CD quality
        bitsize = -16   # unsigned 16 bit
        channels = 2    # 1 is mono, 2 is stereo
        buffer = 1024    # number of samples
        pygame.mixer.init(freq, bitsize, channels, buffer)

    except KeyboardInterrupt:
        while True:
            action = input('Enter Q to Quit, Enter to Skip.').lower()
            if action == 'q':
                pygame.mixer.music.fadeout(1000)
                pygame.mixer.music.stop()
            else:
                break

In [252]:
# Plays music when the cell is executed 

play_music(path_4 + "test_output_8.mid")